In [16]:
c = 3e8  # speed of light (m/s)
fc = 10e9  # Carrier frequency: 10 GHz
wavelength = c / fc
bandwidth = 100e6  # 100 MHz
tx_power_dbm = 30  #  dBm
jam_power_dbm = 50
k = 1.38e-23  # Boltzmann 
GT = 13 # db gain-to-noise-temperature for for 0.33m Equivalent satellite antenna aperturesatellites, or can be 5 dB K^(-1) for 0.13m Equivalent satellite antenna aperture
La = 5 # dB
GT_linear_inv = 10 ** (-GT / 10)
La_linear = 10 ** (La / 10)
noise_power_watt = k * bandwidth * GT_linear_inv * La_linear
Tx_power_watt = 10 ** ((tx_power_dbm  - 30)/ 10)  
jam_power_watt = 10 ** ((jam_power_dbm  - 30)/ 10)

In [17]:
from sionna.rt import Scene, Receiver, Transmitter, PlanarArray, PathSolver
import numpy as np
import vsat_dish_3gpp


def compute_cir(tx_pos, rx_pos, tx_array, rx_array,tx_look_at, rx_look_at, frequency=10e9):
    """
    Compute CIR from a single transmitter to multiple receivers using synthetic arrays.
    
    Args:
        tx_pos:        (3,) list or np.array for transmitter position
        rx_pos_array:  (m,3) array of receiver positions
        tx_array:      PlanarArray for the transmitter
        rx_array:      PlanarArray for all receivers (shared)
        frequency:     Frequency in Hz

    Returns:
        a_list:    list of CIR amplitude arrays, one per RX
        tau_list:  list of delay arrays, one per RX
    """
    scene = Scene()
    scene.frequency = frequency
    scene.synthetic_array = True

    # Add transmitter
    scene.tx_array = tx_array
    tx = Transmitter(name="tx", position=tx_pos, display_radius=200)
    scene.add(tx)

    # Add receivers
    scene.rx_array = rx_array
    rx_list = []
    for i, rx_pos in enumerate(rx_pos):
        rx = Receiver(name=f"rx{i}", position=rx_pos)
        scene.add(rx)
        rx.look_at(rx_look_at)
        rx_list.append(rx)

    tx.look_at(tx_look_at)  # Point TX to the first RX

    # Solve paths
    solver = PathSolver()
    paths = solver(scene=scene,
                   max_depth=0,
                   los=True,
                   synthetic_array=True,
                   seed=41)

    # Get CIRs
    a_all, tau_all = paths.cir(normalize_delays=False, out_type="numpy")

    for tx_name in scene.transmitters:
        scene.remove(tx_name)
    for rx_name in scene.receivers:
        scene.remove(rx_name) 
    
    return a_all, tau_all

jam_rows =8
jam_cols = 8
jam_antennas = jam_cols*jam_rows

sat_rows = 1
sat_cols = 1
sat_antennas = sat_cols*sat_rows

tx_rows = 8
tx_cols = 8
tx_antennas = tx_cols*tx_rows

tx_array = PlanarArray(num_rows=tx_rows, num_cols=tx_cols,
                        vertical_spacing=0.5, horizontal_spacing=0.5,
                        pattern="tr38901", polarization="V")
                        # pattern="iso", polarization="V")

jam_array = PlanarArray(num_rows=jam_rows, num_cols=jam_cols,  
                            vertical_spacing=0.5, horizontal_spacing=0.5,
                        #  pattern="vsat_dish",
                            pattern="tr38901",
                            polarization="V")

sat_array = PlanarArray(num_rows=sat_rows, num_cols=sat_cols,
                             vertical_spacing=0.5, horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")




In [18]:
from scipy.optimize import linprog

def solve_mixed_strategy(A):
    """
    Solve mixed strategy (minimization for column player in zero-sum game).
    A: payoff matrix (numpy array, shape (m, n))
    Returns: optimal probabilities q (for columns) and game value
    """
    m, n = A.shape


    # min t
    # s.t. A q <= t * 1
    #      sum(q) = 1, q >= 0

    c = np.zeros(n + 1)
    c[-1] = 1.0  

    A_ub = np.hstack([A, -np.ones((m, 1))])
    b_ub = np.zeros(m)


    A_eq = np.zeros((1, n + 1))
    A_eq[0, :n] = 1.0
    b_eq = np.array([1.0])

    bounds = [(0, None)] * n + [(None, None)]

    res = linprog(c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method="highs")

    if res.success:
        q = res.x[:n]
        game_value = res.x[-1]
        return q, game_value
    else:
        raise RuntimeError("Linear program did not converge")

In [19]:
def single_beamforming(h):
    return h.conj() / np.linalg.norm(h)

In [20]:
import pandas as pd
# Load and make sure each time block is ordered by Rank (nearest first)
df = pd.read_csv("tracked_satellites_tol2.csv")
df = df.sort_values(["Time", "Rank"], ascending=[True, True]).reset_index(drop=True)

In [21]:

n_steps = df["Time"].nunique()
print("Total steps:", n_steps)
max_steps = 3
print(df.head(max_steps*5))

Total steps: 300
                   Time                  Name Action  Rank  Slant km  \
0   2025-09-09 17:21:02  STARLINK-11674 [DTC]   seed     1     485.2   
1   2025-09-09 17:21:02        STARLINK-34534   seed     2     514.9   
2   2025-09-09 17:21:02        STARLINK-32871   seed     3     528.6   
3   2025-09-09 17:21:02         STARLINK-3567   seed     4     582.1   
4   2025-09-09 17:21:02        STARLINK-30357   seed     5     587.5   
5   2025-09-09 17:21:05  STARLINK-11674 [DTC]   keep     1     475.8   
6   2025-09-09 17:21:05        STARLINK-34534   keep     2     525.5   
7   2025-09-09 17:21:05        STARLINK-32871   keep     3     527.7   
8   2025-09-09 17:21:05         STARLINK-3567   keep     4     581.5   
9   2025-09-09 17:21:05        STARLINK-30357   keep     5     584.0   
10  2025-09-09 17:21:08  STARLINK-11674 [DTC]   keep     1     467.1   
11  2025-09-09 17:21:08        STARLINK-32871   keep     2     527.7   
12  2025-09-09 17:21:08        STARLINK-34534  

In [22]:
from leo_utils import arc_point_on_earth, compute_satellite_intersection_point_enu, compute_az_el_dist
import numpy as np


# Generate RX positions

distances_km = [0.0001]
azimuths_deg = np.linspace(0, 360, len(distances_km), endpoint=False)
gnd_positions = [np.array([0.0, 0.0, 0.0])]

for d_km, az in zip(distances_km, azimuths_deg):
    pos = arc_point_on_earth(d_km, az)
    gnd_positions.append(pos)
gnd_positions = np.array(gnd_positions)

for i, pos in enumerate(gnd_positions):
    print(f"TX{i}(m): {pos}")

TX0(m): [0. 0. 0.]
TX1(m): [ 6.12323400e-18  1.00000000e-01 -7.07323089e-10]


In [23]:
from tqdm import tqdm

per_time = {}
frames = []
k=5
max_steps = 100


# df = df.copy()
# df["Time_dt"] = pd.to_datetime(df["Time"])  

# t_start = "2025-09-09 17:21:00"
# t_end   = "2025-09-09 17:22:00"  
# mask = (df["Time_dt"] >= t_start) & (df["Time_dt"] < t_end)
# df_window = df.loc[mask].sort_values(["Time","Rank"])

# for t, g in df_window.groupby("Time"):



value_TX_steps = []         
value_Jam_steps = []         
step_records = []            
value_TX_uni_steps = []      
value_Jam_uni_steps = [] 
value_TX_greedy_steps = []   
value_Jam_greedy_steps = []



groups = list(df.sort_values(["Time","Rank"]).groupby("Time"))
for step_idx, (t, g) in enumerate(tqdm(groups, desc="Processing steps"), start=1):
    # If more/less than k are present, we still proceed with available rows
    gk = g.sort_values("Rank").head(k).copy()

    # Build sat_positions in ENU (meters)
    sat_positions = gk[["x_East (m)", "y_North (m)", "z_Up (m)"]].to_numpy()  # (K,3)
    names = gk["Name"].tolist()
    K = sat_positions.shape[0]
    

    CP_tx_list, CP_tx_jam_list = [], []
    # w_tx_list, w_jam_list = [], []
    # h_tx_list, h_jam_list = [], []

    for i, pos in enumerate(sat_positions):

        a_tx,  tau_tx  = compute_cir(gnd_positions[0], sat_positions, tx_array, sat_array, sat_positions[i], gnd_positions[0])
        a_jam, tau_jam = compute_cir(gnd_positions[1], sat_positions, tx_array, sat_array, sat_positions[i], gnd_positions[0])


        h_tx  = a_tx[i, :, 0, :, 0, 0].T.squeeze()
        h_jam = a_jam[i, :, 0, :, 0, 0].T.squeeze()
        w_tx  = single_beamforming(h_tx)
        w_jam = single_beamforming(h_jam)

        CG_tx  = np.abs(h_tx.conj().T  @ w_tx )**2
        CG_jam = np.abs(h_jam.conj().T @ w_jam)**2
        CP_tx     = np.log2(1 + Tx_power_watt * CG_tx / noise_power_watt)
        CP_tx_jam = np.log2(1 + Tx_power_watt * CG_tx / (jam_power_watt * CG_jam + noise_power_watt))

        CP_tx_list.append(CP_tx)
        CP_tx_jam_list.append(CP_tx_jam)
        # w_tx_list.append(w_tx);  w_jam_list.append(w_jam)
        # h_tx_list.append(h_tx);  h_jam_list.append(h_jam)

    CP_tx     = np.array(CP_tx_list).reshape(-1, 1)   # (K,1)
    CP_tx_jam = np.array(CP_tx_jam_list).reshape(-1, 1)

    TX_Ut = np.tile(CP_tx, (1, K))
    np.fill_diagonal(TX_Ut, CP_tx_jam.flatten())

    deltaC = (CP_tx - CP_tx_jam).flatten()            # (K,)
    Jam_Ut = np.diag(deltaC)


    p_Jam, value_TX  = solve_mixed_strategy(TX_Ut)    
    p_TX,  value_Jam = solve_mixed_strategy(Jam_Ut)  

    value_TX_steps.append(float(value_TX))
    value_Jam_steps.append(float(value_Jam))
    
    p_uni_tx  = np.full(K, 1.0 / K)
    p_uni_jam = np.full(K, 1.0 / K)
    val_TX_uni  = float(p_uni_tx @ TX_Ut @ p_uni_jam)   # TX （uniform）
    val_Jam_uni = float(p_uni_tx @ Jam_Ut @ p_uni_jam)  # Jam （uniform）
    value_TX_uni_steps.append(val_TX_uni)
    value_Jam_uni_steps.append(val_Jam_uni)
    
    
    jam_target = int(np.argmax(deltaC))   
    p_greedy_jam = np.zeros(K); p_greedy_jam[jam_target] = 1.0
    p_uni_tx     = np.full(K, 1.0 / K)   

    val_TX_greedy  = float(p_uni_tx @ TX_Ut @ p_greedy_jam)   
    val_Jam_greedy = float(p_uni_tx @ Jam_Ut @ p_greedy_jam)  

    value_TX_greedy_steps.append(val_TX_greedy)
    value_Jam_greedy_steps.append(val_Jam_greedy)
    
    step_records.append({
        "step": step_idx,
        "time": t,
        "K": K,
        "value_TX": float(value_TX),
        "value_Jam": float(value_Jam),
        "value_TX_uniform": val_TX_uni,
        "value_Jam_uniform": val_Jam_uni,
        "value_TX_greedy": val_TX_greedy,        
        "value_Jam_greedy": val_Jam_greedy,
        "names": names
    })
    
    
    
    if step_idx >= max_steps:
        break



Processing steps:  12%|█▏        | 36/300 [00:01<00:10, 25.80it/s]/home/sj4025/anaconda3/envs/sionna_gpu/lib/python3.11/site-packages/drjit/ast.py:838: RuntimeWarning: The AST-transforming decorator @drjit.syntax was called more than 1000 times by your program. Since transforming and recompiling Python code is a relatively expensive operation, it should not be used within loops or subroutines. Please move the function to be transformed to the top program level and decorate it there.
  warnings.warn(
/home/sj4025/anaconda3/envs/sionna_gpu/lib/python3.11/site-packages/drjit/ast.py:838: RuntimeWarning: The AST-transforming decorator @drjit.syntax was called more than 1000 times by your program. Since transforming and recompiling Python code is a relatively expensive operation, it should not be used within loops or subroutines. Please move the function to be transformed to the top program level and decorate it there.
  warnings.warn(
Processing steps:  13%|█▎        | 39/300 [00:01<00:10, 

In [24]:
avg_value_TX_opt     = float(np.mean(value_TX_steps))        
avg_value_Jam_opt    = float(np.mean(value_Jam_steps))       
avg_value_TX_unif    = float(np.mean(value_TX_uni_steps))    
avg_value_Jam_unif   = float(np.mean(value_Jam_uni_steps))   
avg_value_TX_greedy  = float(np.mean(value_TX_greedy_steps)) 
avg_value_Jam_greedy = float(np.mean(value_Jam_greedy_steps))

print("Mixed Strategy Channel Capacity/Hz")
print("Average TX (optimal):",   avg_value_TX_opt)
print("Average Jam deg(optimal):",  avg_value_Jam_opt)
print("Both Uniform")
print("Average TX (uniform):",   avg_value_TX_unif)
print("Average Jam deg(uniform):",  avg_value_Jam_unif)
print("TX (uniform) Jam greedy)")
print("Average TX (uniform):",    avg_value_TX_greedy)
print("Average Jam deg(greedy):",   avg_value_Jam_greedy)

Mixed Strategy Channel Capacity/Hz
Average TX (optimal): 4.268465660329349
Average Jam deg(optimal): 1.0635285154491054
Both Uniform
Average TX (uniform): 4.276614609716685
Average Jam deg(uniform): 1.0655657465275594
TX (uniform) Jam greedy)
Average TX (uniform): 4.205871917370141
Average Jam deg(greedy): 1.1363084388741032
